In [5]:
import mlflow, bentoml
mlflow.sklearn.save_model(5, "./my_model")

2023/11/15 23:50:13 WARNING mlflow.sklearn: Model was missing function: predict. Not logging python_function flavor!


In [6]:
bentoml.mlflow.import_model("my_sklearn_model", model_uri="./my_model")

BentoMLException: MLflow model "./my_model" does not support the required python_function flavor